In [1]:
#imports
import pandas as pd
import pandas_ta as ta
import requests
import datetime
from backtesting import Backtest, Strategy
import backtrader as bt

/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/backtesting/_plotting.py:50: UserWarning: Jupyter Notebook detected. Setting Bokeh output to notebook. This may not work in Jupyter clients without JavaScript support (e.g. PyCharm, Spyder IDE). Reset with `backtesting.set_bokeh_output(notebook=False)`.
  warnings.warn('Jupyter Notebook detected. '


Loading BokehJS ...

### My Fucntions

In [2]:
#Function to download data from Tiingo API
def get_tiingo_data(ticker, api, startDate='1995-01-01', endDate='2024-03-25'):
    #call Tiingo API and get ticker data - see Tiingo Docs
    headers = {'Content-Type': 'application/json'}
    r = requests.get(f"https://api.tiingo.com/tiingo/daily/{ticker}/prices?startDate={startDate}&endDate={endDate}&token={api}", headers=headers)
    if r.status_code != 200: #print in case of any error
        print("Error number: ", r.status_code)
    #Create a Pandas DataFrame
    data = pd.DataFrame(r.json())
    # Format date into timestamp
    data['date'] = data['date'].apply(lambda x: datetime.datetime.strptime(x, '%Y-%m-%dT%H:%M:%S.%fZ'))
    #set date as index
    data.set_index('date', inplace=True)
    #Drop cols with non ajusted prices
    data.drop(columns=['close',
                      'high',
                      'low',
                      'open',
                      'divCash',
                      'splitFactor',
                      'adjVolume'], inplace=True)
    #Rename columns names
    data.rename(columns={'adjClose': 'Close',
                        'adjHigh': 'High',
                        'adjLow': 'Low',
                        'adjOpen': 'Open',
                        'volume': 'Volume'}, inplace=True)
    return data

In [3]:
def calculateIndicators(data):
    #Calculate indicators using pandas_ta
    data['ema20'] = ta.ema(data['Close'], length=20)
    data['ema25'] = ta.ema(data['Close'], length=25)
    data['ema35'] = ta.ema(data['Close'], length=35)
    data['ema60'] = ta.ema(data['Close'], length=60)
    data['coppock'] = ta.coppock(data['Close'], length=14)
    data['CMO'] = ta.cmo(data['Close'], length=14)
    data['rsx'] = ta.rsx(data['Close'], length=14) #0 to 100
    data['atr'] = ta.atr(high=data['High'], low=data['Low'], close=data['Close'], length=14)
    data['AO'] = ta.ao(high=data['High'], low=data['Low'])
    data['ChoppinessIndex'] = ta.chop(high=data['High'], low=data['Low'], close=data['Close'])
    data['momentum'] = ta.mom(data['Close'])
    data['ROC'] = ta.roc(data['Close'])
    data = data.dropna()
    return data

#For information about pandas_ta indicators: https://github.com/twopirllc/pandas-ta/blob/main/README.md#volatility-14

### Code

In [4]:
#Load tiingo API from file
with open('tiingoAPI.txt', 'r') as file:
    tiingo_api = file.read().strip()

In [5]:
#Download assets data
ko = get_tiingo_data(ticker='ko', api=tiingo_api)
exon = get_tiingo_data(ticker='xom', api=tiingo_api)
jnj = get_tiingo_data(ticker='jnj', api=tiingo_api)
tesla = get_tiingo_data(ticker='tsla', api=tiingo_api)
ford = get_tiingo_data(ticker='f', api=tiingo_api)
apple = get_tiingo_data(ticker='aapl', api=tiingo_api)
coinbase = get_tiingo_data(ticker='coin', api=tiingo_api)
intel = get_tiingo_data(ticker='intc', api=tiingo_api)
dow = get_tiingo_data(ticker='dow', api=tiingo_api)
jp_morgan = get_tiingo_data(ticker='jpm', api=tiingo_api)
hsbc = get_tiingo_data(ticker='hsbc', api=tiingo_api)
hsbc

,Volume,Close,High,Low,Open
date,,,,,
1999-07-16,231400,18.519110,18.593711,18.387811,18.537014
1999-07-19,158200,18.557903,19.005512,18.352002,18.984624
1999-07-20,79400,18.166990,18.426604,18.128197,18.426604
1999-07-21,75200,18.202799,18.259496,18.017787,18.166990
1999-07-22,33500,18.166990,18.202799,18.089404,18.184894
...,...,...,...,...,...
2024-03-19,1293965,38.500000,38.570000,38.340000,38.370000
2024-03-20,2356977,39.060000,39.060000,38.345000,38.380000
2024-03-21,2476832,39.500000,39.735000,39.380000,39.410000


In [6]:
#calculate assets indicators
ko = calculateIndicators(ko)
exon = calculateIndicators(exon)
jnj = calculateIndicators(jnj)
tesla = calculateIndicators(tesla)
ford = calculateIndicators(ford)
apple = calculateIndicators(apple)
coinbase = calculateIndicators(coinbase)
intel = calculateIndicators(intel)
dow = calculateIndicators(dow)
jp_morgan = calculateIndicators(jp_morgan)
hsbc = calculateIndicators(hsbc)
hsbc

,Volume,Close,High,Low,Open,ema20,ema25,ema35,ema60,coppock,CMO,rsx,atr,AO,ChoppinessIndex,momentum,ROC
date,,,,,,,,,,,,,,,,,
1999-10-08,32500,17.743988,17.783211,17.535803,17.650455,17.798455,17.851702,17.907648,18.111538,-5.132351,-3.646082,43.710766,0.405517,-0.432165,63.807101,0.244391,1.396552
1999-10-11,34200,17.535803,17.614249,17.406064,17.575026,17.773441,17.827402,17.886990,18.092661,-4.041441,-8.830807,44.384732,0.400632,-0.424081,62.155846,-0.491799,-2.728033
1999-10-12,94700,17.234086,17.499597,16.914265,17.499597,17.722074,17.781762,17.850717,18.064511,-4.170830,-15.894268,44.605831,0.416584,-0.454608,56.679818,-0.247408,-1.415257
1999-10-13,159300,16.669875,16.914265,16.500913,16.914265,17.621864,17.696232,17.785115,18.018785,-4.485128,-27.245785,44.025088,0.439429,-0.616417,48.251600,-0.564211,-3.273810
1999-10-14,81900,17.083227,17.083227,16.630651,16.763407,17.570565,17.649078,17.746121,17.988111,-4.103312,-14.999920,43.495914,0.440377,-0.735418,48.825036,-0.434473,-2.480193
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2024-03-19,1293965,38.500000,38.570000,38.340000,38.370000,37.748541,37.710808,37.671418,37.592024,4.117966,15.712319,54.486393,0.564401,0.529299,52.274769,1.100345,2.942126
2024-03-20,2356977,39.060000,39.060000,38.345000,38.380000,37.873442,37.814592,37.748562,37.640154,4.895000,24.069036,56.183185,0.575158,0.654530,44.865176,1.372138,3.640795
2024-03-21,2476832,39.500000,39.735000,39.380000,39.410000,38.028352,37.944239,37.845864,37.701133,5.882012,29.946011,58.320417,0.582289,0.995149,36.034978,1.620000,4.276663


In [7]:
#export to csv to serve as a backup
ko.to_csv("KO.csv", index=True)
exon.to_csv("EXON.csv", index=True)
jnj.to_csv("JNJ.csv", index=True)
tesla.to_csv("TESLA.csv", index=True)
ford.to_csv("FORD.csv", index=True)
apple.to_csv("APPLE.csv", index=True)
coinbase.to_csv("COINBASE.csv", index=True)
intel.to_csv("INTEL.csv", index=True)
dow.to_csv("DOW.csv", index=True)
jp_morgan.to_csv("JPM.csv", index=True)
hsbc.to_csv("HSBC.csv", index=True)

In [8]:
"""#Import data from cvs
ko = pd.read_csv("KO.csv")
exon = pd.read_csv("EXON.csv")
jnj = pd.read_csv("JNJ.csv")
tesla = pd.read_csv("TESLA.csv")
ford = pd.read_csv("FORD.csv")
apple = pd.read_csv("APPLE.csv")
coinbase = pd.read_csv("COINBASE.csv")
intel = pd.read_csv("INTEL.csv")
dow = pd.read_csv("DOW.csv")
jp_morgan = pd.read_csv("JPM.csv")
hsbc = pd.read_csv("HSBC.csv")"""

'#Import data from cvs\nko = pd.read_csv("KO.csv")\nexon = pd.read_csv("EXON.csv")\njnj = pd.read_csv("JNJ.csv")\ntesla = pd.read_csv("TESLA.csv")\nford = pd.read_csv("FORD.csv")\napple = pd.read_csv("APPLE.csv")\ncoinbase = pd.read_csv("COINBASE.csv")\nintel = pd.read_csv("INTEL.csv")\ndow = pd.read_csv("DOW.csv")\njp_morgan = pd.read_csv("JPM.csv")\nhsbc = pd.read_csv("HSBC.csv")'

### Backtest with backtesting.py

In [182]:
class trading_strategy(Strategy):
    
    def init(self):
        self.already_bought = False
        self.stop_loss = 0
        self.take_profit = 0
        
        
    def next(self):
        if (self.data.CMO[-2] > -60 and self.data.CMO[-1] < -60
                and self.already_bought == False):
            self.stop_loss = self.data.Close[-1] - 3 * self.data.atr[-1]
            self.take_profit = self.data.Close[-1] + 5 * self.data.atr[-1]
            self.buy(tp=self.take_profit, sl=self.stop_loss)
            self.already_bought = True
        #elif self.data.Close[-1] < self.stop_loss or self.data.Close[-1] > self.take_profit and self.already_bought == True:
        elif self.position:
            self.already_bought = False
        else:
            pass
#and self.data.Close[-1] > self.data.ema25[-1]

In [183]:
#Run the startegy
#bt = Backtest(exon, trading_strategy, cash=1000, commission=.002, exclusive_orders=True)
bt_exon = Backtest(exon, trading_strategy, cash=1000, commission=.002)
results_exon = bt_exon.run()
results_exon

Start                     1995-03-28 00:00:00
End                       2024-03-25 00:00:00
Duration                  10590 days 00:00:00
Exposure Time [%]                    2.000274
Equity Final [$]                  1469.124038
Equity Peak [$]                   1469.124038
Return [%]                          46.912404
Buy & Hold Return [%]             1613.337598
Return (Ann.) [%]                    1.336929
Volatility (Ann.) [%]                7.390281
Sharpe Ratio                         0.180904
Sortino Ratio                        0.293711
Calmar Ratio                         0.041362
Max. Drawdown [%]                   -32.32238
Avg. Drawdown [%]                   -6.049854
Max. Drawdown Duration       84 days 00:00:00
Avg. Drawdown Duration       19 days 00:00:00
# Trades                                    5
Win Rate [%]                             60.0
Best Trade [%]                      31.375549
Worst Trade [%]                     -7.154833
Avg. Trade [%]                    

In [184]:
bt_exon.plot()
#bt.plot(resample=False)

/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/backtesting/_plotting.py:250: UserWarning: DatetimeFormatter scales now only accept a single format. Using the first provided: '%d %b'
  formatter=DatetimeTickFormatter(days=['%d %b', '%a %d'],
/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/backtesting/_plotting.py:250: UserWarning: DatetimeFormatter scales now only accept a single format. Using the first provided: '%m/%Y'
  formatter=DatetimeTickFormatter(days=['%d %b', '%a %d'],
/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/backtesting/_plotting.py:659: UserWarning: found multiple competing values for 'toolbar.active_drag' property; using the latest value
  fig = gridplot(
/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/backtesting/_plotting.py:659: UserWarning: found multiple competing values for 'toolbar.active_scroll' property; using the latest value
  fi

GridPlot(id='p8531', ...)

In [185]:
#Run the strategy for all assets
bt_ko = Backtest(ko, trading_strategy, cash=1000, commission=.002)
results_ko = bt_ko.run()
bt_jnj = Backtest(jnj, trading_strategy, cash=1000, commission=.002)
results_jnj = bt_jnj.run()
bt_tesla = Backtest(tesla, trading_strategy, cash=1000, commission=.002)
results_tesla = bt_tesla.run()
bt_ford = Backtest(ford, trading_strategy, cash=1000, commission=.002)
results_ford = bt_ford.run()
bt_apple = Backtest(apple, trading_strategy, cash=1000, commission=.002)
results_apple = bt_apple.run()
bt_coinbase = Backtest(coinbase, trading_strategy, cash=1000, commission=.002)
results_coinbase = bt_coinbase.run()
bt_intel = Backtest(intel, trading_strategy, cash=1000, commission=.002)
results_intel = bt_intel.run()
bt_dow = Backtest(dow, trading_strategy, cash=1000, commission=.002)
results_dow = bt_dow.run()
bt_jp_morgan = Backtest(jp_morgan, trading_strategy, cash=1000, commission=.002)
results_jp_morgan = bt_jp_morgan.run()
bt_hsbc = Backtest(hsbc, trading_strategy, cash=1000, commission=.002)
results_hsbc = bt_hsbc.run()

In [186]:
#Create a dataframe with results to better compare
exon_data = results_exon.to_frame(name='exon')
ko_data = results_ko.to_frame(name='ko')
jnj_data = results_jnj.to_frame(name='jnj')
tesla_data = results_tesla.to_frame(name='tesla')
ford_data = results_ford.to_frame(name='ford')
apple_data = results_apple.to_frame(name='apple')
coinbase_data = results_coinbase.to_frame(name='coin')
intel_data = results_intel.to_frame(name='intel')
dow_data = results_dow.to_frame(name='dow')
jpm_data = results_jp_morgan.to_frame(name='jpm')
hsbc_data = results_hsbc.to_frame(name='hsbc')

In [187]:
#concatenate dfs
all_results = pd.concat([
    exon_data,
    ko_data,
    jnj_data,
    tesla_data,
    ford_data,
    apple_data,
    coinbase_data,
    intel_data,
    dow_data,
    jpm_data,
    hsbc_data
], axis=1)

all_results

,exon,ko,jnj,tesla,ford,apple,coin,intel,dow,jpm,hsbc
Start,1995-03-28 00:00:00,1995-03-28 00:00:00,1995-03-28 00:00:00,2010-09-22 00:00:00,1995-03-28 00:00:00,1995-03-28 00:00:00,2021-07-08 00:00:00,1995-03-28 00:00:00,2019-06-13 00:00:00,1995-03-28 00:00:00,1999-10-08 00:00:00
End,2024-03-25 00:00:00,2024-03-25 00:00:00,2024-03-25 00:00:00,2024-03-25 00:00:00,2024-03-25 00:00:00,2024-03-25 00:00:00,2024-03-25 00:00:00,2024-03-25 00:00:00,2024-03-25 00:00:00,2024-03-25 00:00:00,2024-03-25 00:00:00
Duration,10590 days 00:00:00,10590 days 00:00:00,10590 days 00:00:00,4933 days 00:00:00,10590 days 00:00:00,10590 days 00:00:00,991 days 00:00:00,10590 days 00:00:00,1747 days 00:00:00,10590 days 00:00:00,8935 days 00:00:00
Exposure Time [%],2.000274,3.685436,2.808604,2.383054,4.658172,0.232909,56.954612,1.219345,5.980066,5.795314,3.314917
Equity Final [$],1469.124038,748.919784,1124.006673,1904.563379,1255.373735,968.611787,2498.189357,1029.821573,1037.361929,840.449759,773.217201
Equity Peak [$],1469.124038,1076.170567,1262.407686,2277.243379,1255.373735,1277.408155,2498.189357,1029.821573,1157.667689,1958.113793,1129.41468
Return [%],46.912404,-25.108022,12.400667,90.456338,25.537374,-3.138821,149.818936,2.982157,3.736193,-15.955024,-22.67828
Buy & Hold Return [%],1613.337598,767.645477,1900.579015,12931.957725,138.530752,66216.87435,14.499161,1228.979272,39.49981,3641.60248,122.723328
Return (Ann.) [%],1.336929,-0.993241,0.404415,4.892366,0.788311,-0.110045,40.187095,0.101506,0.770694,-0.598315,-1.047662
Volatility (Ann.) [%],7.390281,5.840089,5.897048,10.076525,11.014841,4.723622,122.047442,2.263738,22.58731,13.495888,7.203355


In [188]:
#Print some important results
positives, negatives = sum(1 for c in all_results.iloc[4] if c > 1000), sum(1 for c in all_results.iloc[4] if c <= 1000)
print(f"Number of assets with POSITIVE returns: {positives}")
print(f"Number of assets with NEGATIVE returns: {negatives}")
print(f"Number of total trades: {all_results.iloc[17].sum()}")
print(all_results.iloc[4])
print(f"End portfolio value of {all_results.iloc[4].sum():.2f}$ out of 11000$")

Number of assets with POSITIVE returns: 7
Number of assets with NEGATIVE returns: 4
Number of total trades: 54
exon     1469.124038
ko        748.919784
jnj      1124.006673
tesla    1904.563379
ford     1255.373735
apple     968.611787
coin     2498.189357
intel    1029.821573
dow      1037.361929
jpm       840.449759
hsbc      773.217201
Name: Equity Final [$], dtype: object
End portfolio value of 13649.64$ out of 11000$


### Stategies tested


- MA crossover
- MA and close
- AO indicator
- ChoppinessIndex
- Coppock
- rsx (enter in oversold zone; leave oversold zone; rsx legnth=21)
- CMO (0 cross with and without MA; crossing multiple various negative levels )


### Try strategies